In [ ]:
from torchvision import datasets

In [ ]:
train_dataset=( datasets.MNIST('./data', train=True, download=True,transform=SimCLRDataTransform(
                          transforms.Compose([transforms.RandomResizedCrop(size=28,scale=(0.2,0.8)),
                                              GaussianBlur(kernel_size=int(0.1 * 28)),
                                              transforms.ToTensor(),
                                              transforms.Normalize((0.1307,), (0.3081,))]))))

In [ ]:
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
from gaussian_blur import GaussianBlur

In [ ]:
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

transform_train = transforms.Compose([
    transforms.RandomCrop(28, padding=2),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])   

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])   

train_dataset1 = MNIST('./data', train=True, download=True,transform=transform_train)
test_dataset1 = MNIST('./data', train=False, download=True,transform=transform_test)

train_loader1 = DataLoader(train_dataset1, batch_size=512, shuffle=False, num_workers=0)
test_loader1 = DataLoader(test_dataset1, batch_size=2000, shuffle=False, num_workers=0)

In [ ]:
import struct
import numpy as np

def decode_idx3_ubyte(idx3_ubyte_file):
    with open(idx3_ubyte_file, 'rb') as f:
        fb_data = f.read()
    offset = 0
    fmt_header = '>iiii'    # 以大端法读取4个 unsinged int32
    magic_number, num_images, num_rows, num_cols = struct.unpack_from(fmt_header, fb_data, offset)
    offset += struct.calcsize(fmt_header)
    fmt_image = '>' + str(num_rows * num_cols) + 'B'

    images = np.empty((num_images, num_rows, num_cols))
    for i in range(num_images):
        im = struct.unpack_from(fmt_image, fb_data, offset)
        images[i] = np.array(im).reshape((num_rows, num_cols))
        offset += struct.calcsize(fmt_image)
    return images

In [ ]:
import struct
def decode_idx1_ubyte(idx1_ubyte_file):
    with open(idx1_ubyte_file, 'rb') as f:
        fb_data = f.read()
    offset = 0
    fmt_header = '>ii'  # 以大端法读取两个 unsinged int32
    magic_number, label_num = struct.unpack_from(fmt_header, fb_data, offset)
    offset += struct.calcsize(fmt_header)
    labels = []
    fmt_label = '>B'    # 每次读取一个 byte
    for i in range(label_num):
        labels.append(struct.unpack_from(fmt_label, fb_data, offset)[0])
        offset += struct.calcsize(fmt_label)
    return labels

In [ ]:
import numpy as np
import os
import cv2

def check_folder(folder):
    if not os.path.exists(folder):
        os.mkdir(folder)
        print(folder)
    else:
        if not os.path.isdir(folder):
            os.mkdir(folder)


def export_img(exp_dir, img_ubyte, lable_ubyte):
    check_folder(exp_dir)
    images = decode_idx3_ubyte(img_ubyte)
    labels = decode_idx1_ubyte(lable_ubyte)

    nums = len(labels)
    for i in range(nums):
        img_dir = os.path.join(exp_dir, str(labels[i]))
        check_folder(img_dir)
        img_file = os.path.join(img_dir, str(i)+'.png')
        imarr = images[i]
        cv2.imwrite(img_file, imarr)


def parser_mnist_data(data_dir):
    train_dir = os.path.join(data_dir, 'train')
    train_img_ubyte = os.path.join(data_dir, 'train-images.idx3-ubyte')
    train_label_ubyte = os.path.join(data_dir, 'train-labels.idx1-ubyte')
    export_img(train_dir, train_img_ubyte, train_label_ubyte)

    test_dir = os.path.join(data_dir, 'test')
    test_img_ubyte = os.path.join(data_dir, 't10k-images.idx3-ubyte')
    test_label_ubyte = os.path.join(data_dir, 't10k-labels.idx1-ubyte')
    export_img(test_dir, test_img_ubyte, test_label_ubyte)


if __name__ == '__main__':
    data_dir = 'C:\\Users\\yijia\\Desktop\\mnist_data'
    parser_mnist_data(data_dir)